# Gather data from various source into the analysis repository

In [6]:
# imports
import os
import shutil
import numpy as np
import pandas as pd
import yaml
import git
import tempfile
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

import pint
unit_registry = pint.UnitRegistry()

from PLBenchmarks import targets, ligands, edges
from IPython.core.display import HTML

from tqdm.notebook import tqdm

import benchmarkpl
path = benchmarkpl.__path__[0]

# Set path of data directory

In [7]:
targets.set_data_dir(os.path.join(path))
# directory name where results for each target are stored
results_dir = '10_results'

# Number of targets, ligands and edges in the data set

In [8]:
nligs, nedgs = 0, 0
print(f'{"Target":10s} {"Num Ligs":>10s} {"Num Edges":>10s}')
print(33 * '-')
for target in tqdm(targets.target_dict):
    print(f'{target:10s} {len(ligands.LigandSet(target)):10d} {len(edges.EdgeSet(target)):10d}')
    nligs += len(ligands.LigandSet(target))
    nedgs += len(edges.EdgeSet(target))
print(33 * '-')
print(f'{"total":10s} {nligs:10d} {nedgs:10d}')

Target       Num Ligs  Num Edges
---------------------------------


  0%|          | 0/22 [00:00<?, ?it/s]

jnk1               21         31
pde2               21         34
thrombin           11         16
p38                34         56
ptp1b              23         49
galectin            8          7
cdk2               16         25
cmet               24         57
mcl1               42         71
bace               36         58
bace_hunt          32         60
bace_p2            12         26
tyk2               16         24
ros1               28         61
eg5                28         65
cdk8               33         54
hif2a              42         92
pfkfb3             40         66
pde10              35         59
shp2               26         56
syk                44        101
tnks2              27         60
---------------------------------
total             599       1128


# Gather data from Gapsys et al.

In [9]:
import subprocess
for forcefield in ['openff-2.0.0.offxml']:
    for target in [
               #'bace', 
               'bace_hunt', 
               #'bace_p2', 
               #'cdk2', 
               #'galectin', 
               #'jnk1', 
 #              'mcl1', 
 #              'p38', 
  #             'pde10', 
 #              'pde2', 
 #              'ptp1b', 
#                'ros1', 
 #              'thrombin', 
 #              'tyk2'
              ]:
        print(target)
        # workpath/[water|complex]/edge* - every edge has its own folder
        waterComplex = ['water','complex']
        # workpath/[water|complex]/edge*/state[A|B] - two states will be considered for every edge
        states = ['stateA','stateB']
        
        if forcefield == 'gaff2_sh':
            nFF = 'gaff2-sh'
        else:
            nFF = forcefield
            
        for edge in edges.EdgeSet(target):
            print(edge)
            if target == "pde10" and edge in ["edge_8414_1038",
                         "edge_8414_1056",
                         "edge_8414_2333",
                         "edge_8414_5644"]:
                continue
    #         pmxworkflow.printInfo(runtype='Analyse', run='', target=pwf.target, edge=edge, wc='', state='')
            for wc in waterComplex:
                print(f'    - {wc}')
                for run in [1, 2, 3]:
                    os.makedirs(f'analyze/{forcefield}/{target}/{wc}/{edge}/analyze{run}/', exist_ok=True)

                    # input files
    #                 fa = [f'{pwf.runPath}/{edge}/{wc}/stateA/morphes{run}/dgdl{i}.xvg' for i in range(1, 81)]
    #                 fb = [f'{pwf.runPath}/{edge}/{wc}/stateB/morphes{run}/dgdl{i}.xvg' for i in range(1, 81)]
    #               # output files
                    result = f'analyze/{forcefield}/{target}/{wc}/{edge}/analyze{run}/results.txt'
                    if wc == 'complex':
                        outWorkA = f'gapsys_openff-2.0.0_work/output-run2-protein/{target}/{edge}/stateA/run{run}/work.dat'
                        outWorkB = f'gapsys_openff-2.0.0_work/output-run2-protein/{target}/{edge}/stateB/run{run}/work.dat'
                    elif wc == 'water':
                        outWorkA = f'gapsys_openff-2.0.0_work/output-run2-water/{target}/{edge}/stateA/run{run}/work.dat'
                        outWorkB = f'gapsys_openff-2.0.0_work/output-run2-water/{target}/{edge}/stateB/run{run}/work.dat'
                    else:
                        raise Exception("wrong wc.")
                    # plot = f'{pwf.runPath}/{edge}/{wc}/analyse{run}/plot.png'

                    cmdline = f'pmx analyse '\
                    f'-o {result} '\
                    f'-iA {outWorkA} '\
                    f'-iB {outWorkB} '\
                    f'--work_plot none '\
                    f'-b 100 '\
                    f'-t 298 '\
                    f'-m bar '\
                    f'--no_ks '\
                    f'--unit kcal'

                    print(cmdline)
                    process = subprocess.run(cmdline.split(),
                                           stdout=subprocess.PIPE,
                                           stderr=subprocess.PIPE)
    #                 process.wait()


bace_hunt
edge_33_13
    - water
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_13/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_33_13/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_33_13/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_13/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_33_13/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_33_13/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_13/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_33_13/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_33_13/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_k

pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_28_34/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_28_34/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_28_34/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_28_34/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_28_34/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_28_34/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_28_34/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_28_34/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_28_34/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal

pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_22_16/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_22_16/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_22_16/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_22_16/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_22_16/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_22_16/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_22_16/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_22_16/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_22_16/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --uni

    - complex
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_37_34/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_37_34/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_37_34/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_37_34/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_37_34/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_37_34/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_37_34/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_37_34/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_37_34/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks

pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_35_37/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_35_37/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_35_37/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_35_37/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_35_37/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_35_37/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
edge_21_37
    - water
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_21_37/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_21_37/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_21_37/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no

...

pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_39_14/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_39_14/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_39_14/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_39_14/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_39_14/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_39_14/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
edge_17_34
    - water
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_17_34/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_17_34/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_17_34/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no

pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_23_36/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_23_36/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_23_36/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
edge_11_24
    - water
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_11_24/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_11_24/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_11_24/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_11_24/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_11_24/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_11_24/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --

edge_38_34
    - water
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_38_34/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_38_34/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_38_34/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_38_34/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_38_34/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_38_34/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_38_34/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_38_34/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_38_34/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit k

pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_21_38/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_21_38/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_21_38/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_21_38/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_21_38/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_21_38/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_21_38/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_21_38/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_21_38/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal

pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/water/edge_41_29/analyze3/results.txt -iA gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_41_29/stateA/run3/work.dat -iB gapsys_openff-2.0.0_work/output-run2-water/bace_hunt/edge_41_29/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_41_29/analyze1/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_41_29/stateA/run1/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_41_29/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_41_29/analyze2/results.txt -iA gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_41_29/stateA/run2/work.dat -iB gapsys_openff-2.0.0_work/output-run2-protein/bace_hunt/edge_41_29/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --uni

In [10]:
def read_neq_results( fname ):
    if not os.path.exists(fname):
        print('File does not exist')
        return []
    fp = open(fname,'r')
    lines = fp.readlines()
    fp.close()
    out = []
    for l in lines:
        l = l.rstrip()
        foo = l.split()
        if 'BAR: dG' in l:
            out.append(float(foo[-2]))
        elif 'BAR: Std Err (bootstrap)' in l:
            out.append(float(foo[-2]))
        elif 'BAR: Std Err (analytical)' in l:
            out.append(float(foo[-2]))
        elif 'BAR: Conv' in l:
            out.append(float(foo[-1]))
    return(out)


In [11]:
def output_textfile(edgs, df, fname):
    fp = open(fname, 'w')
    fp.write('#%6s  %6s  %6s\n' % ('1_edge', '2_ddg', '3_ddg_err'))
    fp.write('#all values in kJ/mol\n')

    for i, row in edgs.iterrows():
        edge = f'edge_{row[0]}_{row[1]}'
        val = df.loc[f'{edge}_ddg', 'val']
        err = df.loc[f'{edge}_ddg', 'err']

        fp.write('%10s  %4.2f  %4.2f\n' % (edge, val, err))

    fp.close()


In [12]:
def getGapsysResults(target, forcefield):  
#     file_name = f'{temp_directory}/protLig_benchmark/ddg_data/{target}.dat'
#     if not os.path.exists(file_name):
#         print(f'File {file_name} does not exist.')
#         return None
#     data = pd.read_csv(f'{temp_directory}/protLig_benchmark/ddg_data/{target}.dat', sep='\s+', header=None, comment='#',
#                    names=['edge', 'exp', 'gaff', 'dgaff', 'cgenff', 'dcgenff', 'cons', 'dcons', 'fep5', 'dfep5', 'fep1', 'dfep1'])
#     data['dexp']=pd.Series([0.0]*data.shape[0])
    edgs = edges.EdgeSet(target).get_dataframe()

    bootnum = 1000
    ######## read into a data frame #########
    df = pd.DataFrame()
    arrays = [['water', 'complex', 'ddg'] * 3,
              [int(i / 3) + 1 for i in range(9)],
              [''] * 9]
    arrays = [[] * 9] * 3
    tuples = list(zip(*arrays))

    index = pd.MultiIndex.from_tuples(tuples, names=['leg', 'repeat', ''])
    newdf = pd.DataFrame(columns=index)

    # workpath/[water|complex]/edge* - every edge has its own folder
    waterComplex = ['water','complex']
    waterComplexNew = ['water','complex']
    
    if forcefield == 'openff-2.0.0-rc.1.offxml':
        nFF = 'openff-2.0rc1'
    elif forcefield == 'gaff':
        nFF = 'gaff2_sh'
    else:
        nFF = forcefield
    
    if not os.path.exists(f'analyze/{nFF}/{target}/'):
        return None
    # workpath/[water|complex]/edge*/state[A|B] - two states will be considered for every edge
    states = ['stateA','stateB']
    for i, row in edgs.iterrows():
        edge = f'edge_{row[0]}_{row[1]}'
        for wc, nwc in zip(waterComplex, waterComplexNew):
            for run in range(1,4):
                result = f'analyze/{nFF}/{target}/{wc}/{edge}/analyze{run}/results.txt'
                print(result)
                foo = read_neq_results(result)
                print(foo)
                if len(foo) > 1:
                    df.loc[f'{edge}_{nwc}{run}','val'] = foo[0] #unit_registry.Quantity(foo[0], 'kilojoules/mole').to('kilocalories/mole').magnitude
                    df.loc[f'{edge}_{nwc}{run}','err'] = foo[2] #unit_registry.Quantity(foo[2], 'kilojoules/mole').to('kilocalories/mole').magnitude
                    df.loc[f'{edge}_{nwc}{run}','aerr'] = foo[1] #unit_registry.Quantity(foo[1], 'kilojoules/mole').to('kilocalories/mole').magnitude
                    df.loc[f'{edge}_{nwc}{run}','conv'] = foo[3]
                else:
                    df.loc[f'{edge}_{nwc}{run}','val'] = np.nan
                    df.loc[f'{edge}_{nwc}{run}','err'] = np.nan
                    df.loc[f'{edge}_{nwc}{run}','aerr'] = np.nan
                    df.loc[f'{edge}_{nwc}{run}','conv'] = np.nan
                    print('Results could not be read')
                for t in ['val', 'err', 'aerr', 'conv']:
                    newdf.loc[f'{edge}', (nwc, run, t)] = df.loc[f'{edge}_{nwc}{run}', t]
        vals = []
        errs = []
        aerrs = []
        for run in range(1,4):
            ##### calculate ddg #####
            ddg = df.loc[f'{edge}_complex{run}','val'] - df.loc[f'{edge}_water{run}','val']
            err = np.sqrt( np.power(df.loc[f'{edge}_complex{run}','err'],2.0) +
                           np.power(df.loc[f'{edge}_water{run}','err'],2.0) )
            aerr = np.sqrt( np.power(df.loc[f'{edge}_complex{run}','aerr'],2.0) +
                            np.power(df.loc[f'{edge}_water{run}','aerr'],2.0) )
            df.loc[f'{edge}_ddg{run}','val'] = ddg
            df.loc[f'{edge}_ddg{run}','err'] = err
            df.loc[f'{edge}_ddg{run}','aerr'] = aerr
            newdf.loc[f'{edge}', ('ddg', run, 'val')] = ddg
            newdf.loc[f'{edge}', ('ddg', run, 'err')] = err
            newdf.loc[f'{edge}', ('ddg', run, 'aerr')] = aerr
            vals.append(ddg)
            errs.append(err)
            aerrs.append(aerr)

        ###### calculate mean dg with err ######
        # mean ddg
        mean = np.average(vals)
        df.loc[f'{edge}_ddg', 'val'] = mean

        # error
        # 1) create three distributions
        distribs = []
        for v, e in zip(vals, errs):
            distribs.append(np.random.normal(v, e, size=bootnum))
        if len(distribs) > 1:
            distr = np.vstack(distribs)
            # 2) calculate stderrs
            stderr = np.mean(np.sqrt(np.var(distr, ddof=1, axis=0) / np.float(len(distribs))))
            df.loc[f'{edge}_ddg', 'err'] = stderr
        else:
            stderr = errs[0]
            df.loc[f'{edge}_ddg','err'] = stderr

        for t in ['val', 'err', 'aerr']:
            newdf.loc[f'{edge}', ('ddg_mean', '-', t)] = df.loc[f'{edge}_ddg', t]
    ###### output ######
    output_textfile(edgs, df, os.path.join(path, '..', '00_data', 'input', f'{target}_{forcefield}.dat'))

    newdf.to_csv(os.path.join(path, '..', '00_data', 'input', f'{target}_{forcefield}.csv'), float_format='%10.2f')


#     newdata = data.copy()
#     newdata.index=newdata['edge']
#     newdata.columns = pd.MultiIndex.from_arrays([np.array(newdata.columns), ['', 'exp'] + ['pmx'] * 6 + ['fep'] * 4 + ['exp'], [''] + ['kj/mol'] * 12], names=['forcefield', 'method', 'unit'])
    
#     newdata.loc[:,('ligandA', '', '')] = df[0].apply(lambda x: str(int(float(x))) if str(x).endswith('.0') else x).astype(str)
#     newdata.loc[:,('ligandB', '', '')] = df[1].apply(lambda x: str(int(float(x))) if str(x).endswith('.0') else x).astype(str)
#     newdata = newdata.drop(columns=('edge', '', ''))
#     newdata.sort_index(axis=1, level=1, inplace=True, sort_remaining=False)
#     newdata.index=[f'{target}_edge_{x[("ligandA", "", "")]}_{x[("ligandB", "", "")]}' for i, x in newdata.iterrows()]
    
#     return newdata
# getGapsysResults('cmet')

In [13]:
# function to retrieve data from PLBenchmarks calculations, soted in 00_data/input 
def getRawResults(target, forcefield='openff-1.0.0.offxml'):
    file_path = os.path.join(path, 
                                   '..', 
                                   '00_data', 
                                   'input',  
                                   f'{target}_{forcefield}.dat'
                                  )
    if not os.path.exists(file_path):
        print(f'File {file_path} does not exist.')
        return
    # read in result file
    res = pd.read_csv(file_path,
                      header=None, 
                      comment='#', 
                      skipinitialspace=True, 
                      names=['edge', 'calc DDG', 'calc dDDG' , 'add dDDG'], 
                      sep=' ')
    res.index = res['edge']
    # read in exp. data
    edg = edges.EdgeSet(target)
    df = edg.get_dataframe(columns=[0,1, 'exp. DeltaG [kcal/mol]', 'exp. Error [kcal/mol]'])
    df.index = pd.Series(['edge_' + str(lig1) + '_' + str(lig2) for lig1, lig2 in zip(df[0].values, df[1].values)])
   
    # copy in calculated values
    df['calc_DDG'] = res.loc[:,'calc DDG']
    df['calc_dDDG'] = res.loc[:,'calc dDDG']
    df['calc_dDDG(additional)'] = res.loc[:, 'add dDDG']
    
    # remove unit of calculated values
    df['exp_DDG'] = df['exp. DeltaG [kcal/mol]'].apply(lambda x: x.magnitude)
    df['exp_dDDG'] = df['exp. Error [kcal/mol]'].apply(lambda x: x.magnitude)

    # filter only to relevant columns
    df = df.filter(items = [0,1,'exp_DDG', 'exp_dDDG', 'calc_DDG', 'calc_dDDG', 'calc_dDDG(additional)'])
    df[0]= df[0].astype(str)
    df[1]= df[1].astype(str)
    df.index = pd.Series([f'{target}_edge_' + str(lig1) + '_' + str(lig2) for lig1, lig2 in zip(df[0].values, df[1].values)])
    return df

In [14]:
author = "gapsys"
software = "pmx"
for target in [
#     'bace', 
                'bace_hunt',
#                'bace_p2',
#                'cdk2',
#                'galectin',
#                'jnk1',
#                'mcl1',
#                'p38',
#                'pde10',
#                'pde2',
#                'ptp1b',
#                'ros1',
#                'thrombin',
#                'tyk2'
              ]:
    print(target)
    for forcefield in ['openff-2.0.0.offxml']:
        print(f"    {forcefield}")
        
        getGapsysResults(target, forcefield)
        df = getRawResults(target, forcefield)
        if df is None:
            print('None')
            continue
        os.makedirs(os.path.join(path, targets.get_target_dir(target), results_dir), exist_ok=True)
        with open(os.path.join(path, targets.get_target_dir(target), results_dir, f'{target}_{software}_{forcefield}_{author}.yaml'), 'w') as file:
            df = df.filter([0,1,'calc_DDG', 'calc_dDDG'])
            df.rename(columns={0: 'ligandA', 1: 'ligandB', 'calc_DDG': 'DDG', 'calc_dDDG': 'dDDG'}, inplace=True)
            df['unit']='kilocalories / mole'
            yaml.dump(df.T.to_dict(), file)

bace_hunt
    openff-2.0.0.offxml
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_13/analyze1/results.txt
[32.21, 0.05, 0.06, -0.05, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_13/analyze2/results.txt
[32.09, 0.06, 0.06, -0.02, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_13/analyze3/results.txt
[32.18, 0.05, 0.03, 0.03, 0.03]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_33_13/analyze1/results.txt
[30.34, 0.11, 0.08, 0.19, 0.08]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_33_13/analyze2/results.txt
[30.25, 0.1, 0.12, -0.24, 0.06]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_33_13/analyze3/results.txt
[29.78, 0.1, 0.09, 0.1, 0.05]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_25/analyze1/results.txt
[28.31, 0.06, 0.06, 0.03, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_25/analyze2/results.txt
[28.42, 0.06, 0.06, -0.03, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_33_25/analyze3/results.txt
[28.43, 0.06, 0.05,

[-7.04, 0.21, 0.22, -0.06, 0.11]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_1a_37/analyze2/results.txt
[-8.08, 0.14, 0.12, 0.08, 0.07]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_1a_37/analyze3/results.txt
[-7.69, 0.17, 0.15, 0.06, 0.09]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_27_37/analyze1/results.txt
[-18.02, 0.05, 0.04, 0.01, 0.03]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_27_37/analyze2/results.txt
[-17.91, 0.05, 0.06, -0.04, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_27_37/analyze3/results.txt
[-18.02, 0.05, 0.06, -0.02, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_27_37/analyze1/results.txt
[-15.86, 0.11, 0.09, 0.2, 0.08]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_27_37/analyze2/results.txt
[-15.37, 0.07, 0.08, -0.18, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_27_37/analyze3/results.txt
[-15.78, 0.1, 0.09, 0.12, 0.06]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_35_37/analyze1/results.txt
[4.99, 0.0

[18.58, 0.1, 0.1, 0.04, 0.07]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_31_13/analyze3/results.txt
[18.1, 0.1, 0.12, -0.05, 0.06]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_38_31/analyze1/results.txt
[-22.16, 0.04, 0.04, 0.04, 0.03]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_38_31/analyze2/results.txt
[-22.21, 0.05, 0.04, 0.0, 0.03]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_38_31/analyze3/results.txt
[-22.18, 0.05, 0.05, 0.02, 0.03]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_38_31/analyze1/results.txt
[-21.29, 0.08, 0.07, 0.06, 0.05]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_38_31/analyze2/results.txt
[-21.5, 0.09, 0.06, 0.24, 0.05]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_38_31/analyze3/results.txt
[-21.47, 0.08, 0.07, 0.14, 0.05]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_24_1a/analyze1/results.txt
[-34.35, 0.13, 0.13, 0.01, 0.08]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_24_1a/analyze2/results.txt
[-34.58, 0.13, 0.1

[74.87, 0.29, 0.25, 0.06, 0.19]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_39_36/analyze3/results.txt
[73.85, 0.34, 0.36, -0.05, 0.2]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_23_36/analyze1/results.txt
[24.96, 0.06, 0.06, -0.02, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_23_36/analyze2/results.txt
[24.96, 0.06, 0.06, -0.02, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_23_36/analyze3/results.txt
[25.02, 0.06, 0.05, 0.0, 0.04]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_23_36/analyze1/results.txt
[23.68, 0.08, 0.08, -0.1, 0.05]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_23_36/analyze2/results.txt
[23.53, 0.08, 0.09, -0.08, 0.05]
analyze/openff-2.0.0.offxml/bace_hunt/complex/edge_23_36/analyze3/results.txt
[23.37, 0.1, 0.1, 0.05, 0.05]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_11_24/analyze1/results.txt
[17.46, 0.06, 0.06, 0.03, 0.03]
analyze/openff-2.0.0.offxml/bace_hunt/water/edge_11_24/analyze2/results.txt
[17.48, 0.06, 0.06, 